## TYTANチュートリアル（ナンプレ＆不等号ナンプレ）

2023年5月7日

ビネクラ安田

出典：[量子アニーリングでナンプレ（数独）と不等号ナンプレを解く](https://vigne-cla.com/21-19/)


### 問題
ナンバープレース（数独）の亜種である不等号ナンプレを解く。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-19_1-300x300.png" width = 18%>
</div>

答えはこちら。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-19_2-300x300.png" width = 18%>
</div>


### QUBOモデルでは何が設定できるか？（おさらい）

**<font color="red">One-hot表現</font>**

例）1～3の自然数のどれかにする（＝3個の量子ビットから1個を１にする。1になった場所を自然数に割り当てる）
```
H += (q0 + q1 + q2 - 1)**2
```


**<font color="red">「n個の量子ビットからm個を1にする」</font>**

例）ある量子ビットを1にする（＝１個の量子ビットから１個を１にする）
```
H = (q0 - 1)**2
```

**<font color="red">「2個の量子ビットが同時に1になったら報酬を与える」</font>**

例）2個の量子ビットが同時に1になったら報酬を与える
```
H = -(q0 * q1)
```

けっこう盛りだくさんで難しい。その他の条件式も気になる方は → [量子アニーリングのQUBOで設定可能な条件式まとめ（保存版）](https://vigne-cla.com/21-12/)

## 通常のナンプレの条件設定

4×4のナンプレでは4×4×4＝64個の量子ビットを用意する。4個セットで一つの数字を表すワンホット表現とし、次のように3次元的な配置を想像する。今回、混乱を避けるために量子ビット名を１始まりにしているが、かえって分かりにくいかもしれない。例えば、q112は1行目1列目のワンホット2番。これが１であればそのマスの数字は２となる。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-19_3-768x522.png" width = 55%>
</div>

まずワンホットの設定として、「各ワンホット(ｋの次元)は1つだけ1になる」を16回設定する。あとは、「各行(ｊの次元)は1つだけ1になる」を16回、「各列(ｉの次元)は1つだけ1になる」を16回設定すれば「各行、各列の数字が重複しない」となり通常のナンプレのルールを満たす。要するに、**3つの次元方向すべてについて「1つだけ1になる」**とする。

初めから記入されている数字については、その部分のワンホットのどこが1になるか指定しておく（「ある量子ビットを1にする」）。これで解が絞れる。

### 通常のナンプレのコード

左上マスを２で固定するだけの通常のナンプレを試す。式が多いためfor文使ってしまってごめんなさい。printされた式を見て理解を深めてください。

In [ ]:
!pip install git+https://github.com/tytansdk/tytan

In [2]:
from sympy import Symbol
from tytan import qubo, sampler
import numpy as np

#量子ビットを用意する
#１始まり！！！！！！！！！！！！！！！
for i in range(1, 5):
    for j in range(1, 5):
        for k in range(1, 5):
            command = f'q{i}{j}{k} = Symbol(\'q{i}{j}{k}\')'
            print(command)
            exec(command)

#各マスはワンホットで一つだけ１になる（kの次元のワンホット）
H = 0
for i in range(1, 5):
    for j in range(1, 5):
        command = f'H += (q{i}{j}1 + q{i}{j}2 + q{i}{j}3 + q{i}{j}4 - 1)**2'
        print(command)
        exec(command)

#各行で数字が重複しない（つまり各行は一つだけ１になる）（jの次元のワンホット）
for i in range(1, 5):
    for k in range(1, 5):
        command = f'H += (q{i}1{k} + q{i}2{k} + q{i}3{k} + q{i}4{k} - 1)**2'
        print(command)
        exec(command)

#各列で数字が重複しない（つまり各列は一つだけ１になる）（iの次元のワンホット）
for j in range(1, 5):
    for k in range(1, 5):
        command = f'H += (q1{j}{k} + q2{j}{k} + q3{j}{k} + q4{j}{k} - 1)**2'
        print(command)
        exec(command)

#数字指定のマス
H += (q112 - 1)**2
print('H += (q112 - 1)**2')


#コンパイル
QUBO, offset = qubo.Compile(H).get_qubo()

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(QUBO, shots=500)

#上位3件をワンホットから整数に戻して確認
for r in result[:3]:
    print(r)

    box = np.array(list(r[0].values())).reshape(4, 4, 4)
    ans = np.zeros((4, 4), int)
    for i in range(4):
        for j in range(4):
            ans[i, j] = np.argmax(box[i, j, :]) + 1
    print(ans)

q111 = Symbol('q111')
q112 = Symbol('q112')
q113 = Symbol('q113')
q114 = Symbol('q114')
q121 = Symbol('q121')
q122 = Symbol('q122')
q123 = Symbol('q123')
q124 = Symbol('q124')
q131 = Symbol('q131')
q132 = Symbol('q132')
q133 = Symbol('q133')
q134 = Symbol('q134')
q141 = Symbol('q141')
q142 = Symbol('q142')
q143 = Symbol('q143')
q144 = Symbol('q144')
q211 = Symbol('q211')
q212 = Symbol('q212')
q213 = Symbol('q213')
q214 = Symbol('q214')
q221 = Symbol('q221')
q222 = Symbol('q222')
q223 = Symbol('q223')
q224 = Symbol('q224')
q231 = Symbol('q231')
q232 = Symbol('q232')
q233 = Symbol('q233')
q234 = Symbol('q234')
q241 = Symbol('q241')
q242 = Symbol('q242')
q243 = Symbol('q243')
q244 = Symbol('q244')
q311 = Symbol('q311')
q312 = Symbol('q312')
q313 = Symbol('q313')
q314 = Symbol('q314')
q321 = Symbol('q321')
q322 = Symbol('q322')
q323 = Symbol('q323')
q324 = Symbol('q324')
q331 = Symbol('q331')
q332 = Symbol('q332')
q333 = Symbol('q333')
q334 = Symbol('q334')
q341 = Symbol('q341')
q342 = Sym

左上の２を指定しただけなので非常に多くの解が得られる。たいていの問題はもっと数字が指定されていて、解が一意に定まるように作られている。それについては各自で試してほしい。

## 不等号ナンプレの条件設定

不等号ナンプレではさらに条件設定を追加する。不等号関係にある量子ビット８個に着目すると

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-19_4r-768x205.png" width = 65%>
</div>

小も大もワンホットなので1つだけ1が立つが、**大の方はより右側のビットが立たなければならない。**よって、「赤線のペアが同時に1になったときに報酬を与える」という設定でそれを促してやる。赤線のペアは6通りあるので6式になる。

（補足）ここまでの条件設定はすべて重み1.0で設定する。重みに差をつけるのは「優先したいこと」がある場合、つまり、叶わないものがある場合。今回はすべての設定が叶う解が存在することがわかっているので重みを付ける必要がない（むしろつけてしまうとおかしな事が起きる）。

### 不等号ナンプレのコード

条件式のprintはコメントアウトした。

In [8]:
from sympy import Symbol
from tytan import qubo, sampler
import numpy as np

#量子ビットを用意する
#１始まり！！！！！！！！！！！！！！！
for i in range(1, 5):
    for j in range(1, 5):
        for k in range(1, 5):
            command = f'q{i}{j}{k} = Symbol(\'q{i}{j}{k}\')'
            #print(command)
            exec(command)

#各マスはワンホットで一つだけ１になる（kの次元のワンホット）
H = 0
for i in range(1, 5):
    for j in range(1, 5):
        command = f'H += (q{i}{j}1 + q{i}{j}2 + q{i}{j}3 + q{i}{j}4 - 1)**2'
        #print(command)
        exec(command)

#各行で数字が重複しない（つまり各行は一つだけ１になる）（jの次元のワンホット）
for i in range(1, 5):
    for k in range(1, 5):
        command = f'H += (q{i}1{k} + q{i}2{k} + q{i}3{k} + q{i}4{k} - 1)**2'
        #print(command)
        exec(command)

#各列で数字が重複しない（つまり各列は一つだけ１になる）（iの次元のワンホット）
for j in range(1, 5):
    for k in range(1, 5):
        command = f'H += (q1{j}{k} + q2{j}{k} + q3{j}{k} + q4{j}{k} - 1)**2'
        #print(command)
        exec(command)

#数字指定のマス
H += (q112 - 1)**2
#print('H += (q112 - 1)**2')

#不等号
H += -(q121 * q222)
H += -(q121 * q223)
H += -(q121 * q224)
H += -(q122 * q223)
H += -(q122 * q224)
H += -(q123 * q224)

H += -(q141 * q242)
H += -(q141 * q243)
H += -(q141 * q244)
H += -(q142 * q243)
H += -(q142 * q244)
H += -(q143 * q244)

H += -(q311 * q212)
H += -(q311 * q213)
H += -(q311 * q214)
H += -(q312 * q213)
H += -(q312 * q214)
H += -(q313 * q214)

H += -(q321 * q332)
H += -(q321 * q333)
H += -(q321 * q334)
H += -(q322 * q333)
H += -(q322 * q334)
H += -(q323 * q334)


#コンパイル
QUBO, offset = qubo.Compile(H).get_qubo()

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(QUBO, shots=1000)

#上位3件をワンホットから整数に戻して確認
for r in result[:3]:
    print(r)

    box = np.array(list(r[0].values())).reshape(4, 4, 4)
    ans = np.zeros((4, 4), int)
    for i in range(4):
        for j in range(4):
            ans[i, j] = np.argmax(box[i, j, :]) + 1
    print(ans)

[{'q111': 0.0, 'q112': 1.0, 'q113': 0.0, 'q114': 0.0, 'q121': 0.0, 'q122': 0.0, 'q123': 1.0, 'q124': 0.0, 'q131': 0.0, 'q132': 0.0, 'q133': 0.0, 'q134': 1.0, 'q141': 1.0, 'q142': 0.0, 'q143': 0.0, 'q144': 0.0, 'q211': 0.0, 'q212': 0.0, 'q213': 1.0, 'q214': 0.0, 'q221': 0.0, 'q222': 0.0, 'q223': 0.0, 'q224': 1.0, 'q231': 1.0, 'q232': 0.0, 'q233': 0.0, 'q234': 0.0, 'q241': 0.0, 'q242': 1.0, 'q243': 0.0, 'q244': 0.0, 'q311': 1.0, 'q312': 0.0, 'q313': 0.0, 'q314': 0.0, 'q321': 0.0, 'q322': 1.0, 'q323': 0.0, 'q324': 0.0, 'q331': 0.0, 'q332': 0.0, 'q333': 1.0, 'q334': 0.0, 'q341': 0.0, 'q342': 0.0, 'q343': 0.0, 'q344': 1.0, 'q411': 0.0, 'q412': 0.0, 'q413': 0.0, 'q414': 1.0, 'q421': 1.0, 'q422': 0.0, 'q423': 0.0, 'q424': 0.0, 'q431': 0.0, 'q432': 1.0, 'q433': 0.0, 'q434': 0.0, 'q441': 0.0, 'q442': 0.0, 'q443': 1.0, 'q444': 0.0}, -53.0, 1]
[[2 3 4 1]
 [3 4 1 2]
 [1 2 3 4]
 [4 1 2 3]]
[{'q111': 0.0, 'q112': 1.0, 'q113': 0.0, 'q114': 0.0, 'q121': 0.0, 'q122': 0.0, 'q123': 1.0, 'q124': 0.0, 'q13

エネルギー＝-53が最適解であり、模範解答の通りである。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-19_2-300x300.png" width = 18%>
</div>